### A2: Bias in data

### Getting the article and population data

In [1]:
## getting the data from the CSV files
import csv




In [2]:
# reading data and saving in dictionary of dictionaries
'''
{
    "revision_id": {
        "country": "",
        "article_name": "",
        "revision_id":"",
    },
    "revision_id": {
    
    },
    ...
}
'''

page_data = dict()
skip_lines = 1

with open('page_data.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        #page_data.append([row[0],row[1],row[2]])
        #['Template:ZambiaProvincialMinisters', 'Zambia', '235107991']
        if(skip_lines!=1):
            revision_id = row[2]
            page_data[revision_id]=dict()
            page_data[revision_id]["country"] = row[1]
            page_data[revision_id]["article_name"] = row[0]
            page_data[revision_id]["revision_id"] = row[2] 
            
        skip_lines = skip_lines+1
print(len((page_data)))

47197


In [3]:

'''
{
    "country_name_1":"population",
    "country_name_1":"population"
}

'''
population_data = {}

# skip twolines from input csv file 
skip_lines = 1
with open('Population Mid-2015.csv', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # last line in the raw data file is an empty line
        # skip_lines > 2 skips first two lines
        # row checks if the read list is empty
        #print(row)
        if(skip_lines>3 and row):
            # ['Afghanistan', 'Country', 'Mid-2015', 'Number', '32,247,000', '']
            population_data[row[0]] = row[4]
        skip_lines = skip_lines + 1
        
print(population_data)

{'Afghanistan': '32,247,000', 'Albania': '2,892,000', 'Algeria': '39,948,000', 'Andorra': '78,000', 'Angola': '25,000,000', 'Antigua and Barbuda': '90,000', 'Argentina': '42,426,000', 'Armenia': '3,017,106', 'Australia': '23,888,000', 'Austria': '8,615,955', 'Azerbaijan': '9,651,000', 'Bahamas': '377,000', 'Bahrain': '1,412,299', 'Bangladesh': '160,411,000', 'Barbados': '278,000', 'Belarus': '9,524,247', 'Belgium': '11,211,064', 'Belize': '368,000', 'Benin': '10,583,034', 'Bhutan': '757,000', 'Bolivia': '10,475,500', 'Bosnia-Herzegovina': '3,650,114', 'Botswana': '2,139,900', 'Brazil': '204,519,398', 'Brunei': '413,000', 'Bulgaria': '7,181,000', 'Burkina Faso': '18,450,400', 'Burundi': '10,742,000', 'Cambodia': '15,417,100', 'Cameroon': '23,739,000', 'Canada': '35,833,000', 'Cape Verde': '514,000', 'Central African Republic': '5,551,900', 'Chad': '13,707,000', 'Channel Islands': '164,000', 'Chile': '18,025,000', 'China': '1,371,920,000', 'Colombia': '48,218,000', 'Comoros': '764,000', 

In [4]:
# merging data set from wikipedia data(page_data) and population data(population_data)
# reading data from page_data and including poulation data
'''
{
    "revision_id": {
        "country": "",
        "article_name": "",
        "revision_id":"",
        "population":""
    },
    "revision_id": {
    
    },
    ...
}
'''
count=0
for key,value in page_data.items():
    revision_id = key
    #print(key)
    
    country_page_data = value["country"]
    #print(country_page_data)
    
    if population_data.get(str(country_page_data)) is not None:
        page_data[revision_id]["population"] = population_data[country_page_data]
    else:
        # for now setting population as 0 for countries which are not preset in population_data
        page_data[revision_id]["population"] = 0
        count = count+1
        
print("no of entries which have popluation  = 0")      
print(count)



no of entries which have popluation  = 0
1398


### Getting article quality predictions

In [23]:
import requests
import json

headers = {'User-Agent' : 'https://github.com/abhishekanand', 'From' : 'anand1@uw.edu'}

def get_ores_data(revision_ids, headers):
    
    # Define the endpoint
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
    
    # Specify the parameters - smushing all the revision IDs together separated by | marks.
    # Yes, 'smush' is a technical term, trust me I'm a scientist.
    # What do you mean "but people trusting scientists regularly goes horribly wrong" who taught you tha- oh.  
    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join(str(x) for x in revision_ids)
              }
    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    json.dumps(response, indent=4, sort_keys=True)
    #print(response)
    return response


# So if we grab some example revision IDs and turn them into a list and then call get_ores_data...
#example_ids = [783381498, 807355596, 757539710]
#get_ores_data(example_ids, headers)


#get_ores_data(example_ids, headers)
no_revision_ids = len(page_data)
print(no_revision_ids)

# getting all the revision ids from page_data in list format
revision_ids = list(page_data.keys())


counter = 0
call_count = 0

# this contains revsion id as key and article quality as value
data_revision_quality  = {}



# prediction is being added in page_data
'''
{
    "revision_id": {
        "country": "",
        "article_name": "",
        "revision_id":"",
        "population":"",
        "article_quality":""
    },
    "revision_id": {
    
    },
    ...
}
'''

while(counter<no_revision_ids):
    temp = get_ores_data(revision_ids[counter:counter+100], headers)
    hundred_revisions = temp["enwiki"]["scores"]
    
    for key, value in hundred_revisions.items():
        revision = key
        if value["wp10"].get("score") is not None:
            prediction = value["wp10"]["score"]["prediction"]
            page_data[revision]["article_quality"] = prediction
            #print(page_data[revision])
        else:
            page_data[revision]["article_quality"] = 'NA'
        
        
    counter = counter + 100
    call_count = call_count + 1
    
    
if(counter>no_revision_ids):
    temp = get_ores_data(revision_ids[counter-100:(counter-100)+no_revision_ids%100], headers)
    left_revisions = temp["enwiki"]["scores"]
    
    counter = counter-100
    call_count = call_count + 1
    
    for key, value in left_revisions.items():
        revision = key
        counter = counter + 1
        if value["wp10"].get("score") is not None:
            prediction = value["wp10"]["score"]["prediction"]
            page_data[revision]["article_quality"] = prediction
            #print(page_data[revision])

    

47197


In [24]:
print(counter)
print(call_count)
print(no_revision_ids%100)

47197
473
97


In [25]:
# getting page_data into required csv format

combined_data = []
for key, value in page_data.items():
    combined_data.append(value)
    
'''
printing first 100 revision ids because of 
"OPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`."
'''
for i in range(100):    
    print(combined_data[i])
    

{'country': 'Zambia', 'article_name': 'Template:ZambiaProvincialMinisters', 'revision_id': '235107991', 'population': '15,473,900', 'article_quality': 'Stub'}
{'country': 'Chad', 'article_name': 'Bir I of Kanem', 'revision_id': '355319463', 'population': '13,707,000', 'article_quality': 'Stub'}
{'country': 'Zimbabwe', 'article_name': 'Template:Zimbabwe-politician-stub', 'revision_id': '391862046', 'population': '17,354,000', 'article_quality': 'Stub'}
{'country': 'Uganda', 'article_name': 'Template:Uganda-politician-stub', 'revision_id': '391862070', 'population': '40,141,000', 'article_quality': 'Stub'}
{'country': 'Namibia', 'article_name': 'Template:Namibia-politician-stub', 'revision_id': '391862409', 'population': '2,482,100', 'article_quality': 'Stub'}
{'country': 'Nigeria', 'article_name': 'Template:Nigeria-politician-stub', 'revision_id': '391862819', 'population': '181,839,400', 'article_quality': 'Stub'}
{'country': 'Colombia', 'article_name': 'Template:Colombia-politician-st

In [31]:
# The final data file should be named: en-wikipedia_traffic_200801-201709.csv
import csv
CSVOut = "bias_in_data.csv"
with open(CSVOut, "w",encoding="utf-8") as ofile:
    ofile.write("country" + "," + "article_name" + "," + "revision_id"+"," + "article_quality"+"," + "population"+ "\n")
    for item in combined_data:
        #print(item["article_quality"])
        #print(str(item["population"]).replace(',',''))
        ofile.write(str(item["country"]).replace(',','') + "," + str(item["article_name"]).replace(',','') + "," + str(item["revision_id"])+"," + str(item["article_quality"])+"," + str(item["population"]).replace(',','') + "\n")
print("Done")    

Done


### Analysis

FA - Featured article
GA - Good article
B - B-class article
C - C-class article
Start - Start-class article
Stub - Stub-class article

####  if a country has a population of 10,000 people, and you found 10 articles about politicians from that country, then the percentage of articles-per-population would be .1%.

In [59]:
# Creating Country and Article Count  
'''
{
    "country_name_1":"Count of Articles",
    "country_name_1":"Count of Articles"
}

'''

country_articleCount = {}

for item in combined_data:
    country_name = str(item["country"]).replace(',','')
    if (country_name in country_articleCount.keys()):
        if(item["article_name"]) is not None:
            country_articleCount[country_name]=country_articleCount[country_name]+1
    else:
        country_articleCount[country_name]=0
        if(item["article_name"]) is not None:
            country_articleCount[country_name]=1
        
print(country_articleCount)

{'Zambia': 26, 'Chad': 100, 'Zimbabwe': 167, 'Uganda': 188, 'Namibia': 165, 'Nigeria': 684, 'Colombia': 288, 'Chile': 352, 'Fiji': 199, 'Solomon Islands': 98, 'Palestinian Territory': 183, 'Somalia': 339, 'Cambodia': 217, 'Slovakia': 119, 'Slovenia': 59, 'Afghanistan': 327, 'Iraq': 302, 'Nepal': 363, 'Sri Lanka': 465, 'Laos': 109, 'Albania': 460, 'Costa Rica': 150, 'Hondura': 189, 'Czech Republic': 254, 'Canada': 852, 'Tunisia': 140, 'Guatemala': 84, 'Burkina Faso': 97, 'Angola': 110, 'Panama': 109, 'Japan': 441, 'Indonesia': 215, 'Madagascar': 240, 'Malaysia': 391, 'Gabon': 103, 'Germany': 703, 'Liberia': 158, 'Ghana': 395, 'Peru': 354, 'Argentina': 496, 'Spain': 881, 'South Africa': 382, 'Egypt': 239, 'Nicaragua': 116, 'Bangladesh': 324, 'India': 990, 'Iran': 832, 'Philippines': 515, 'Turkey': 353, 'Austria': 340, 'Azerbaijan': 182, 'Haiti': 166, 'Greece': 311, 'Hungary': 614, 'Iceland': 206, 'Moldova': 426, 'Romania': 348, 'Poland': 809, 'Luxembourg': 180, 'Denmark': 291, 'Kenya': 3

In [57]:
# Population Data 
# print(population_data)

{'Afghanistan': '32,247,000', 'Albania': '2,892,000', 'Algeria': '39,948,000', 'Andorra': '78,000', 'Angola': '25,000,000', 'Antigua and Barbuda': '90,000', 'Argentina': '42,426,000', 'Armenia': '3,017,106', 'Australia': '23,888,000', 'Austria': '8,615,955', 'Azerbaijan': '9,651,000', 'Bahamas': '377,000', 'Bahrain': '1,412,299', 'Bangladesh': '160,411,000', 'Barbados': '278,000', 'Belarus': '9,524,247', 'Belgium': '11,211,064', 'Belize': '368,000', 'Benin': '10,583,034', 'Bhutan': '757,000', 'Bolivia': '10,475,500', 'Bosnia-Herzegovina': '3,650,114', 'Botswana': '2,139,900', 'Brazil': '204,519,398', 'Brunei': '413,000', 'Bulgaria': '7,181,000', 'Burkina Faso': '18,450,400', 'Burundi': '10,742,000', 'Cambodia': '15,417,100', 'Cameroon': '23,739,000', 'Canada': '35,833,000', 'Cape Verde': '514,000', 'Central African Republic': '5,551,900', 'Chad': '13,707,000', 'Channel Islands': '164,000', 'Chile': '18,025,000', 'China': '1,371,920,000', 'Colombia': '48,218,000', 'Comoros': '764,000', 